In [ ]:
def fgsm(x, y_adv, epsilon):
    loss_func = tf.keras.losses.BinaryCrossentropy()
    with tf.GradientTape() as gt:
        gt.watch(x)
        label = model(x)
        loss = loss_func(y_adv, label)
        outputloss = loss.numpy()
        #print (loss)
        
    grad = gt.gradient(loss, x)
    gamma = epsilon * tf.sign(grad)
    
    return gamma, outputloss

In [ ]:
def BIM(x, y_adv, eps):
  alpha = eps/10
  itrs = 0
  while 1:
    noise, loss = fgsm(x, y_adv,alpha)
    #noise = tf.clip_by_value(x-noise-img, -eps, eps)
    # print(noise)
    x = x - noise
    # print(x)
    #preds = new_model.predict(x)
    #print(preds)
    itrs+=1
    if loss<2 or itrs>=10:
      break
  return x, itrs

In [ ]:
def create_perturb_bim(image_name, fromfolder, tofolder, yval):

  old_img = cv2.imread(fromfolder+'/'+image_name)

  old_img = preprocessing_image(old_img)

  preds = model.predict(old_img)
  print("The prediction of the image before attack: ", preds)


  y_adv_label = yval
  
  y_adv = tf.reshape(y_adv_label, shape=[1, preds.shape[-1]])
 
  print(y_adv)
  x_adv, itrs = BIM(old_img, y_adv, 1.4)
  # print(x_adv)
  # noise = x_adv - old_img
  
  newpreds = model.predict(x_adv)
  print("The prediction of the image after %d iterations of BIM attack: %f" % ( itrs, newpreds[0][0]))

  save_img(tofolder+'/'+image_name, x_adv[0])

In [ ]:
images = os.listdir('traffic/')
i=0
for img in images:
  i+=1
  print(str(i)+". "+img)
  create_perturb_bim(img, 'traffic/', 'BIM_WBA_traffic_perturbed/traffic', 0)